In [11]:
import numpy as np
import pandas as pd
import yfinance as yf
from statsmodels.tsa.arima.model import ARIMA
from arch import arch_model
import warnings
warnings.filterwarnings("ignore")

In [12]:
!pip install arch

In [13]:
df = yf.download('SBIN.NS', start = '2015-01-01')
df

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2015-01-01,312.450012,315.000000,310.700012,314.000000,285.500427,6138488
2015-01-02,314.350006,318.299988,314.350006,315.250000,286.636993,9935094
2015-01-05,316.250000,316.799988,312.100006,312.750000,284.363892,9136716
2015-01-06,310.000000,311.100006,298.700012,299.899994,272.680206,15329257
2015-01-07,300.000000,302.549988,295.149994,300.149994,272.907501,15046745
...,...,...,...,...,...,...
2024-07-08,859.750000,865.000000,853.500000,856.250000,856.250000,10359800
2024-07-09,859.900024,869.950012,856.000000,861.299988,861.299988,13585081
2024-07-10,860.099976,860.250000,843.500000,849.000000,849.000000,17343236


In [14]:
df.drop(df.tail(1).index, inplace=True)

# Display the DataFrame after removing the last row
print("\nAfter removing the last row:")
print(df.tail())


After removing the last row:
                  Open        High     Low       Close   Adj Close    Volume
Date                                                                        
2024-07-05  838.000000  861.250000  837.00  859.750000  859.750000  24873722
2024-07-08  859.750000  865.000000  853.50  856.250000  856.250000  10359800
2024-07-09  859.900024  869.950012  856.00  861.299988  861.299988  13585081
2024-07-10  860.099976  860.250000  843.50  849.000000  849.000000  17343236
2024-07-11  854.400024  858.299988  849.75  856.700012  856.700012   8501366


In [15]:
model_arima = ARIMA(df['Close'], order = (5,1,0))
model_arima_fit = model_arima.fit()
arima_pred = model_arima_fit.forecast(steps=1)

In [16]:
arima_pred

2350    857.123788
dtype: float64

In [17]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
from arch import arch_model

# Fit ARIMA model for 'Close' prices
model_arima = ARIMA(df['Close'], order=(5, 1, 0))
model_arima_fit = model_arima.fit()

# Residuals
residuals = pd.DataFrame(model_arima_fit.resid)

# Fit GARCH model to residuals
model_garch = arch_model(residuals, vol='Garch', p=1, q=1)
model_garch_fit = model_garch.fit(disp='off')

# Get GARCH predictions (variance forecast)
garch_pred = model_garch_fit.forecast(start=0).variance.iloc[-1, :][0]

# Fit ARIMA model for 'High' prices
model_arima_high = ARIMA(df['High'], order=(5, 1, 0))
model_arima_fit_high = model_arima_high.fit()

# Fit ARIMA model for 'Low' prices
model_arima_low = ARIMA(df['Low'], order=(5, 1, 0))
model_arima_fit_low = model_arima_low.fit()

# Get ARIMA predictions for 'High' and 'Low' prices
arima_pred_high = model_arima_fit_high.forecast(steps=1)
arima_pred_low = model_arima_fit_low.forecast(steps=1)

# Output predictions
print("Next day Close price prediction (GARCH variance):", arima_pred)
print("Next day High price prediction:", arima_pred_high)
print("Next day Low price prediction:", arima_pred_low)


Next day Close price prediction (GARCH variance): 2350    857.123788
dtype: float64
Next day High price prediction: 2350    860.56954
dtype: float64
Next day Low price prediction: 2350    851.521626
dtype: float64


In [18]:
df.tail()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2024-07-05,838.000000,861.250000,837.00,859.750000,859.750000,24873722
2024-07-08,859.750000,865.000000,853.50,856.250000,856.250000,10359800
2024-07-09,859.900024,869.950012,856.00,861.299988,861.299988,13585081
2024-07-10,860.099976,860.250000,843.50,849.000000,849.000000,17343236
2024-07-11,854.400024,858.299988,849.75,856.700012,856.700012,8501366
